In [ ]:

import os
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np
import spacy
from collections import Counter
import copy
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords

In [ ]:
# If loading does not work, uncomment the below command
#!python -m spacy download en
import en_core_web_sm
nlp = en_core_web_sm.load()

2022-11-29 05:10:06.528450: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 4.1 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
# import sys
# !{sys.executable} -m pip install spacy
# !{sys.executable} -m spacy download en

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2022-11-29 05:10:29.868785: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.4.1/en_core_web_sm-3.4.1-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
import nltk
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Copy of enron.csv")
df = df.drop(["Email Folder"], axis=1)
df

,Unnamed: 0,Author,Folder,File,Message ID,Text,Raw Text
0,0,John,arnold-j,36.0,33491127.1075857594966.,Hey:\nHaven't had the best of months. Like yo...,Message-ID: <33491127.1075857594966.JavaMail.e...
1,1,John,arnold-j,667.0,6384662.1075857656041.,Thank you.\n\n,Message-ID: <6384662.1075857656041.JavaMail.ev...
2,2,John,arnold-j,759.0,21884118.1075857658063.,a couple of observations from here:\ncash/futu...,Message-ID: <21884118.1075857658063.JavaMail.e...
3,3,John,arnold-j,313.0,11352651.1075857600972.,"Frank:\nThe $5,000,000 extra VAR disappears in...",Message-ID: <11352651.1075857600972.JavaMail.e...
4,4,John,arnold-j,710.0,25732708.1075857656969.,don't care about the front. i think its vulne...,Message-ID: <25732708.1075857656969.JavaMail.e...
...,...,...,...,...,...,...,...
113343,119,"Scholtes, Diana",scholtes-d,51.0,14805735.1075840025496.,I received your message and those times are go...,Message-ID: <14805735.1075840025496.JavaMail.e...
113344,120,"Scholtes, Diana",scholtes-d,18.0,20865574.1075840024696.,Our Purchases:\n\n11/1\tTo PSE-EPMI MC - you h...,Message-ID: <20865574.1075840024696.JavaMail.e...
113345,122,"Scholtes, Diana",scholtes-d,8.0,16491778.1075840024450.,1\n\n,Message-ID: <16491778.1075840024450.JavaMail.e...
113346,123,"Scholtes, Diana",scholtes-d,124.0,17311779.1075840027415.,"Stewart, \n \nI left you an envelope with info...",Message-ID: <17311779.1075840027415.JavaMail.e...


In [ ]:
top_authors = df.value_counts(["Folder"])[:5]
df = df.loc[df["Folder"].isin(list(top_authors.index.get_level_values(0)))].drop(["Unnamed: 0"], axis=1).reset_index(drop=True)
df = df[df["Text"]!=" "]
df = df[df["Text"]!="\n"]
df = df.dropna()
df
# df[df["Message ID"]=="204093.1075861733067."]

,Author,Folder,File,Message ID,Text,Raw Text
0,Vince J,kaminski-v,2191.0,9763257.1075856555112.,"Pierre-Philippe,\n\nI have contacted Allison B...",Message-ID: <9763257.1075856555112.JavaMail.ev...
1,Vince J,kaminski-v,1693.0,31194175.1075856543039.,"Martin,\n\nCan you, please, call Shu and provi...",Message-ID: <31194175.1075856543039.JavaMail.e...
2,Vince J,kaminski-v,1952.0,4743153.1075856549060.,UPS Tracking numbers\n,Message-ID: <4743153.1075856549060.JavaMail.ev...
3,Vince J,kaminski-v,4054.0,8413088.1075856988030.,"Steve,\n\nThe schedule is fine with me. One co...",Message-ID: <8413088.1075856988030.JavaMail.ev...
4,Vince J,kaminski-v,1023.0,20177981.1075856527141.,"Zimin,\n\nCan you interview this guy? Maybe Ta...",Message-ID: <20177981.1075856527141.JavaMail.e...
...,...,...,...,...,...,...
28101,"Mann, Kay",mann-k,209.0,25629648.1075861733495.,Getting warmer. Could you send me an email wi...,Message-ID: <25629648.1075861733495.JavaMail.e...
28102,"Mann, Kay",mann-k,142.0,1737650.1075861732004.,It is unclear to me that the coop has the auth...,Message-ID: <1737650.1075861732004.JavaMail.ev...
28103,"Mann, Kay",mann-k,124.0,5911678.1075855241024.,"I would like to take Dec 26, 31 and Jan. 2 as ...",Message-ID: <5911678.1075855241024.JavaMail.ev...
28104,"Mann, Kay",mann-k,74.0,11550877.1075855239931.,"At the risk of asking a stupid question, are w...",Message-ID: <11550877.1075855239931.JavaMail.e...


In [ ]:
df["Folder"].value_counts()

mann-k          8167
kaminski-v      5926
dasovich-j      4805
germany-c       4571
shackleton-s    4003
Name: Folder, dtype: int64

In [ ]:
def uniform_distribution(samples_per_author, df):
    df3 = pd.DataFrame(columns=["Author", "Folder", "File", "Text", "Raw Text"]) 
    for folder in df["Folder"].value_counts().index:
        df3 = df3.append(df[df["Folder"]==folder].sample(n=samples_per_author), ignore_index=True)
    return df3


In [ ]:
df = uniform_distribution(4000, df)
print(df["Folder"].value_counts())
df


mann-k          4000
kaminski-v      4000
dasovich-j      4000
germany-c       4000
shackleton-s    4000
Name: Folder, dtype: int64


,Author,Folder,File,Text,Raw Text,Message ID
0,Kay,mann-k,1554.0,Please print and put with other stuff you just...,Message-ID: <11319742.1075845955870.JavaMail.e...,11319742.1075845955870.
1,Kay,mann-k,2692.0,OPPS. I registered twice. I'm just taking up...,Message-ID: <6446758.1075845983982.JavaMail.ev...,6446758.1075845983982.
2,Kay,mann-k,1904.0,"Hi Sheila,\n\nI'm not up on this part of the d...",Message-ID: <31988618.1075845964396.JavaMail.e...,31988618.1075845964396.
3,Kay,mann-k,1993.0,Here's my stab at it:\n\n,Message-ID: <28515105.1075846076799.JavaMail.e...,28515105.1075846076799.
4,Kay,mann-k,3494.0,"Hi Karen,\n\nI'm looking at a version which wa...",Message-ID: <29986588.1075846114096.JavaMail.e...,29986588.1075846114096.
...,...,...,...,...,...,...
19995,Sara,shackleton-s,2369.0,Kevin: This is set to close tomorrow (so far!...,Message-ID: <13622728.1075844575814.JavaMail.e...,13622728.1075844575814.
19996,Sara,shackleton-s,1278.0,Jeff: I have not heard a word from credit. W...,Message-ID: <25204828.1075844548975.JavaMail.e...,25204828.1075844548975.
19997,"Shackleton, Sara",shackleton-s,87.0,\nScott: this has not been reviewed for accu...,Message-ID: <4926010.1075858807724.JavaMail.ev...,4926010.1075858807724.
19998,Sara,shackleton-s,2238.0,fyi\n,Message-ID: <16414307.1075844572728.JavaMail.e...,16414307.1075844572728.


In [ ]:
#Feature Extraction 
#extract feature - email length in characters. Exclude all whitespace. 
def feature_email_length_characters(text):
    if type(text) == str:
        text = text.strip()
        text = re.sub(r"\W", "", text)
        length = len(text)
        return length
    else:
        return np.NaN

#extract digit density - ratio of number of digits to number of characters
def feature_digit_density(text):
    if type(text) == str:
        text = text.strip()
        text = re.sub(r"\W", "", text)
        total_length = len(text)
        number_digits = len(re.findall(r"\d", text))
        try:
            return (number_digits/total_length)
        except ZeroDivisionError as e:
            return 0
    if type(text) == float:
        return np.NaN

#extract space density - ratio of space to number of characters
def feature_space_density(text):
    if type(text) == str:
        number_space = len(re.findall(r"[\s\n\t]", text))
        text = text.strip()
        text = re.sub(r"\W", "", text)
        total_length = len(text)
        try:
            return (number_space/total_length)
        except ZeroDivisionError as e:
            return 0
    if type(text) == float:
        return np.NaN

#extract number of paragraphs
def feature_paragraph(text):
    if type(text) == str:
        number_paragraphs = len(re.findall(r"\n\n", text))
        return max(1.0, number_paragraphs)
    if type(text) == float:
        return np.NaN
    
# extract number of sentences in paragraphs
def feature_average_characters_paragraph(text):
    if type(text) == str:
        number_paragraphs = len(re.findall(r"\n\n", text))
        if not number_paragraphs:
            return max(1, len(re.findall(r"[.?!]\W", text)))
        else:
            paragraphs = re.findall(r"(?:.+\n)+\n", text)
            length = 0
            for paragraph in paragraphs:
                length += max(1, len(re.findall(r"[.?!]\W", paragraph)))
            return length/number_paragraphs
    if type(text) == float:
        return np.NaN
    

#extract farewell words
def feature_farewell_words(text):
    if type(text) == str:
        try:
            words = text.split()
            for word in reversed(words):
                if re.search(r"\w+", word):
                    last_word = re.search(r"\w+", word).group().lower()
                    return last_word
                else:
                    continue
            return "\n"
        except IndexError as e:
            return np.NaN
    if type(text) == float:
        return np.NaN

def feature_freq_farewell_words(farewell_words, text):
    if type(text) == str:
        if text in farewell_words:
            return text
        else:
            return "Other"
    if type(text) == float:
        return np.NaN
        

#extract last punctuation
def feature_ending_punctuation(text):
    if type(text) == str:
        if re.search(r"[\!\,\.\?\:\'\"]", text):
            try:
                last_punc = re.findall(r"[\!\,\.\?\:\'\"]", text)[-1]
                return last_punc
            except IndexError as e:
                last_punc = len(re.findall(r"[\!\,\.\?\:\'\"]", text))
                if len == 1:
                    return last_punc[0]
        else:
            return "None"
    if type(text) == float:
        return np.NaN

#extract most common used punctuation in the email
def feature_most_used_punctuation(text):
    if type(text) == str:
        if re.search(r"[\!\,\.\?\:\'\"]", text):
            punc = re.findall(r"[\!\,\.\?\:\'\"]", text)
            most_used_punc = Counter(punc).most_common(1)[0][0]
            return most_used_punc
        else:
            return "None"
    if type(text) == float:
        return np.NaN
    
#extract subjectivity and polarity
def feature_subjectivity(text):
    if type(text) == str:
        blob = TextBlob(text)
        return blob.sentiment.subjectivity
    if type(text) == float:
        return np.NaN

#extract polarity
def feature_polarity(text):
    if type(text) == str:
        blob = TextBlob(text)
        return blob.sentiment.polarity
    if type(text) == float:
        return np.NaN

def feature_most_pos(text):
    if type(text) == str:
        blob = TextBlob(text)
        final_pos = []
        for word, pos in blob.tags:
            if word not in stopwords.words("english"):
                final_pos.append(pos)
        count_pos = Counter(final_pos)
        if count_pos.most_common():
            return count_pos.most_common()[0][0]
        else:
            return "Other"
    if type(text) == float:
        return np.NaN

def feature_get_greeting(text):
    if type(text) == str:
        if re.match(r"^\w+", text):
            greeting_word = re.match(r"^\w+", text).group()
            return greeting_word
        else:
            return "None"
    if type(text) == float:
        return np.NaN

def feature_most_common_word(text):
    if type(text) == str:
        blob = TextBlob(text)
        words = []
        for word, pos in blob.tags:
            words.append(word)
        count_word = Counter(words)
        if count_word.most_common():
            return count_word.most_common()[0][0]
        else:
            return np.NaN
    if type(text) == float:
        return np.NaN

def feature_freq_most_common_word(text):
    if type(text) == str:
        blob = TextBlob(text)
        words = []
        for word, pos in blob.tags:
            words.append(word)
        count_word = Counter(words)
        if count_word.most_common():
            return count_word.most_common()[0][1]
        else:
            return 0
    if type(text) == float:
        return np.NaN

def feature_number_words(text):
    if type(text) == str:
        blob = TextBlob(text)
        return len(blob.words)
    if type(text) == float:
        return np.NaN


In [ ]:
#Feature Extraction 
#Average length of words
def avg_length(text):
    if type(text) == str:
        list1 = text.split()
        word_len = 0
        for text in list1:
            strip_text=text.strip()
            if re.search(r"\w+",strip_text):
                strip_text = re.search(r"\w+",strip_text).group()
                word_len += len(strip_text)
        try:
            avg_word_len = word_len/len(list1)
        except ZeroDivisionError as e:
            return np.NaN
        return avg_word_len
    else:
        return np.NaN

#Average Sentence Length
def avg_sentence_length(text):
    if type(text) == str:
        list1 = re.findall(r"[^\.\?\!]+",text)
        sent_len = 0 
        for le in list1:
            sent_len += len(le)
        try:
            return sent_len/len(list1) 
        except:
            return np.NaN      
    else:
        return np.NaN

#Number of short words to overall number of words
def feature_short_word_ratio(text):
    if type(text) == str:
        list1 = text.split()
        short_word = 0
        if len(list1) >= 1:
            for word in list1:
                strip_text = word.strip()
                if re.search(r"\w+",strip_text):
                    strip_text = re.search(r"\w+",strip_text).group()
                    word_len = len(strip_text)
                    if word_len < 4:
                        short_word +=1
            try:
                short_word_rat = short_word/len(list1)
                return short_word_rat
            except:
                return 0
        else:
            return 0  
    else:
        return np.NaN

#Frequency of punctuation
def punctuation_frequency(text):
    if type(text) == str:
        if re.search(r"[\!\,\.\?\:\'\"]", text):
            punc = re.findall(r"[\!\,\.\?\:\'\"]", text)
            freq_punc = len(punc)
            return freq_punc
        else:
            return 0
    if type(text) == float:
        return np.NaN

#Punctuation after greeting
def punctuation_greeting(text):
    if type(text) == str:
        if re.search(r"^\w+([\,\:\?\!\-])\n", text):
            punc = re.search(r"^\w+([\,\:\?\!\-])\n", text).group(1)
            return punc
        else:
            return "None"
    if type(text) == float:
        return np.NaN

def feature_number_special_characters(text):
    if type(text) == str:
        special_characters = re.findall(r"[\@\#\$\%\^\&\~\`\*\(\)\<\>\\\[\]\{\}\|]", text)
        return len(special_characters)
    if type(text) == float:
        return np.NaN

def feature_max_special_character(text):
    if type(text) == str:
        special_characters = re.findall(r"[\@\#\$\%\^\&\~\`\*\(\)\<\>\\\[\]\{\}\|]", text)
        special_char_count = Counter(special_characters)
        if special_char_count.most_common():
            max_special_char = special_char_count.most_common()[0][0]
            return max_special_char
        else:
            return "None"
    if type(text) == float:
        return np.NaN

def feature_freq_max_special_character(text):
    if type(text) == str:
        special_characters = re.findall(r"[\@\#\$\%\^\&\~\`\*\(\)\<\>\\\[\]\{\}\|]", text)
        special_char_count = Counter(special_characters)
        if special_char_count.most_common():
            freq_max_special_char = special_char_count.most_common()[0][1]
            return freq_max_special_char
        else:
            return "0"
    if type(text) == float:
        return np.NaN


In [ ]:
#Feature Extraction 
  
def check_single_sentence(clean_text):
    if type(clean_text) == str:
        ending_punc = re.findall(r"[.?!]", clean_text)
        ending_punc_count = Counter(ending_punc)
        single_sentence = False
        if(ending_punc_count):
            max_ep_char = max(ending_punc_count, key=ending_punc_count.get)
            max_ep_value = max(ending_punc_count.values())
        else:
            max_ep_char = ''
            max_ep_value = 0
        if max_ep_value<=1:
            single_sentence = True
    elif type(clean_text) == float:
        return np.NaN 
    return single_sentence



In [ ]:
email_length = df["Text"].apply(lambda row: feature_email_length_characters(row))
email_length.dropna()

0         51
1         48
2         37
3         15
4        110
        ... 
19995     95
19996     56
19997    122
19998      3
19999     25
Name: Text, Length: 20000, dtype: int64

In [ ]:
digit_density = df["Text"].apply(lambda row: feature_digit_density(row))
digit_density.dropna()

0        0.000000
1        0.000000
2        0.000000
3        0.000000
4        0.018182
           ...   
19995    0.000000
19996    0.000000
19997    0.188525
19998    0.000000
19999    0.000000
Name: Text, Length: 20000, dtype: float64

In [ ]:
space_density = df["Text"].apply(lambda row: feature_space_density(row))
space_density.dropna()

0        0.235294
1        0.312500
2        0.378378
3        0.400000
4        0.309091
           ...   
19995    0.305263
19996    0.339286
19997    0.237705
19998    0.333333
19999    0.200000
Name: Text, Length: 20000, dtype: float64

In [ ]:
number_paragraphs = df["Text"].apply(lambda row: feature_paragraph(row))
number_paragraphs.dropna()

0        1.0
1        1.0
2        2.0
3        1.0
4        2.0
        ... 
19995    1.0
19996    1.0
19997    1.0
19998    1.0
19999    1.0
Name: Text, Length: 20000, dtype: float64

In [ ]:
average_sentences_paragraph = df["Text"].apply(lambda row: feature_average_characters_paragraph(row))
average_sentences_paragraph.dropna()

0        1.0
1        3.0
2        1.0
3        1.0
4        2.0
        ... 
19995    4.0
19996    2.0
19997    1.0
19998    1.0
19999    1.0
Name: Text, Length: 20000, dtype: float64

In [ ]:
farewell_words = df["Text"].apply(lambda row: feature_farewell_words(row))
raw_freq_farewell_words = farewell_words.value_counts()
print(raw_freq_farewell_words)
raw_freq_farewell_words = list(raw_freq_farewell_words[raw_freq_farewell_words>20].index)
print(raw_freq_farewell_words)
freq_farewell_words = []
for word in raw_freq_farewell_words:
    tokens = nlp(word)
    for token in tokens:
        if token.pos_ not in ["PROPN"]:
            freq_farewell_words.append(token.text)

farewell_words = farewell_words.apply(lambda row: feature_freq_farewell_words(freq_farewell_words, row))
farewell_words.dropna()

vince           2676
sara            1566
kay             1485
jeff            1155
thanks           768
                ... 
organization       1
pair               1
invite             1
doo                1
prepay             1
Name: Text, Length: 2822, dtype: int64
['vince', 'sara', 'kay', 'jeff', 'thanks', 'best', 'fyi', 'ss', '853', 'you', 'it', 'fax', 'http', 'me', 'kaminski', 'ckm', 'this', 'the', 'to', 'today', 'questions', 'now', 'know', 'tomorrow', '713', '\n', 'week', 'time', 'is', 'that', 'think', 'day', 'please', 'a', 'too', 'call', 'comments', 'do', '3', 'for', 'yes', 'on', 'there', 'out', 'meeting', 'yet', 'good', 'help', 'up', 'in', '4', '415', 'well', 'much', 'deal', 'deserved', 'ok', 'i', 'one', '646', 'them', 'and', 'work', 'weekend', '2', 'of', 'soon', '1', 'agreement', 'monday', 'again', 'though', 'number', 'email', 'here', 'him', 'be', 'contract', 'man', 'done', '10', 'mail', 'morning', 'about', 'list', 'am', 'later', 'go']


0        thanks
1           day
2           ckm
3            it
4           now
          ...  
19995     Other
19996    please
19997       853
19998       fyi
19999     Other
Name: Text, Length: 20000, dtype: object

In [ ]:
greeting_words = df["Text"].apply(lambda row: feature_get_greeting(row))
greeting_words.dropna()

0        Please
1          OPPS
2            Hi
3          Here
4            Hi
          ...  
19995     Kevin
19996      Jeff
19997      None
19998       fyi
19999       Can
Name: Text, Length: 20000, dtype: object

In [ ]:
most_common_word = df["Text"].apply(lambda row: feature_most_common_word(row))
most_common_word.dropna()

0        Please
1             I
2            Hi
3          Here
4             I
          ...  
19995        to
19996      Jeff
19997     Scott
19998       fyi
19999       Can
Name: Text, Length: 19946, dtype: object

In [ ]:
most_common_word[most_common_word.isna()==True]
df.iloc[1714]


Author                                                      Kay
Folder                                                   mann-k
File                                                      284.0
Text                       I'll discuss with Carlos.\n\nckm\n\n
Raw Text      Message-ID: <12140027.1075846035048.JavaMail.e...
Message ID                              12140027.1075846035048.
Name: 1714, dtype: object

In [ ]:
subjectivity = df["Text"].apply(lambda row: feature_subjectivity(row))
polarity = df["Text"].apply(lambda row: feature_polarity(row))
print(subjectivity.dropna())
print(polarity.dropna())

0        0.287500
1        0.000000
2        0.000000
3        0.000000
4        1.000000
           ...   
19995    0.511111
19996    0.000000
19997    1.000000
19998    0.000000
19999    0.000000
Name: Text, Length: 20000, dtype: float64
0        0.037500
1        0.000000
2        0.000000
3        0.000000
4        0.000000
           ...   
19995    0.191667
19996    0.000000
19997   -0.750000
19998    0.000000
19999    0.000000
Name: Text, Length: 20000, dtype: float64


In [ ]:
freq_most_common_word = df["Text"].apply(lambda row: feature_freq_most_common_word(row))
freq_most_common_word.dropna()

0        1
1        2
2        1
3        1
4        2
        ..
19995    2
19996    1
19997    1
19998    1
19999    1
Name: Text, Length: 20000, dtype: int64

In [ ]:
pos = df["Text"].apply(lambda row: feature_most_pos(row))
pos


0         NN
1        PRP
2         NN
3         RB
4        NNP
        ... 
19995     NN
19996     NN
19997    NNP
19998     NN
19999     MD
Name: Text, Length: 20000, dtype: object

In [ ]:
pos.dropna()

0         NN
1        PRP
2         NN
3         RB
4        NNP
        ... 
19995     NN
19996     NN
19997    NNP
19998     NN
19999     MD
Name: Text, Length: 20000, dtype: object

In [ ]:
last_punc = df["Text"].apply(lambda row: feature_ending_punctuation(row))
last_punc.dropna()

0           .
1           !
2           .
3           :
4           .
         ... 
19995       .
19996       ?
19997       :
19998    None
19999       ?
Name: Text, Length: 20000, dtype: object

In [ ]:
freq_punc = df["Text"].apply(lambda row: feature_most_used_punctuation(row))
freq_punc.dropna()


0           .
1           .
2           ,
3           '
4           ,
         ... 
19995       .
19996       :
19997       :
19998    None
19999       ?
Name: Text, Length: 20000, dtype: object

In [ ]:
avg_len = df["Text"].apply(lambda row: avg_length(row))
avg_len


0        4.636364
1        3.916667
2        3.000000
3        2.800000
4        3.758621
           ...   
19995    3.958333
19996    3.733333
19997    5.130435
19998    3.000000
19999    5.000000
Name: Text, Length: 20000, dtype: float64

In [ ]:
avg_sent_len = df["Text"].apply(lambda row: avg_sentence_length(row))
avg_sent_len

0         31.500000
1         16.000000
2         26.500000
3         23.000000
4         37.250000
            ...    
19995     25.400000
19996     25.333333
19997    158.000000
19998      4.000000
19999     15.000000
Name: Text, Length: 20000, dtype: float64

In [ ]:
short_word_ratio = df["Text"].apply(lambda row: feature_short_word_ratio(row))
short_word_ratio.dropna()

0        0.272727
1        0.416667
2        0.666667
3        0.600000
4        0.586207
           ...   
19995    0.416667
19996    0.400000
19997    0.304348
19998    1.000000
19999    0.200000
Name: Text, Length: 20000, dtype: float64

In [ ]:
punc_freq = df["Text"].apply(lambda row: punctuation_frequency(row))
punc_freq

0        1
1        4
2        3
3        2
4        8
        ..
19995    5
19996    3
19997    4
19998    0
19999    1
Name: Text, Length: 20000, dtype: int64

In [ ]:
punc_greet = df["Text"].apply(lambda row: punctuation_greeting(row))
punc_greet.dropna()

0        None
1        None
2        None
3        None
4        None
         ... 
19995    None
19996    None
19997    None
19998    None
19999    None
Name: Text, Length: 20000, dtype: object

In [ ]:
number_words = df["Text"].apply(lambda row: feature_number_words(row))
number_words

0        11
1        13
2        13
3         6
4        30
         ..
19995    24
19996    15
19997    23
19998     1
19999     5
Name: Text, Length: 20000, dtype: int64

In [ ]:
number_special_characters = df["Text"].apply(lambda row: feature_number_special_characters(row))
print(number_special_characters.dropna())
max_special_character = df["Text"].apply(lambda row: feature_max_special_character(row))
print(max_special_character.dropna())
freq_max_special_character = df["Text"].apply(lambda row: feature_freq_max_special_character(row))
print(freq_max_special_character.dropna())

single_sentence = df["Text"].apply(lambda row: check_single_sentence(row))
print(single_sentence.dropna())

0        0
1        0
2        0
3        0
4        0
        ..
19995    2
19996    0
19997    2
19998    0
19999    0
Name: Text, Length: 20000, dtype: int64
0        None
1        None
2        None
3        None
4        None
         ... 
19995       (
19996    None
19997       (
19998    None
19999    None
Name: Text, Length: 20000, dtype: object
0        0
1        0
2        0
3        0
4        0
        ..
19995    1
19996    0
19997    1
19998    0
19999    0
Name: Text, Length: 20000, dtype: object
0         True
1        False
2         True
3         True
4        False
         ...  
19995    False
19996     True
19997     True
19998     True
19999     True
Name: Text, Length: 20000, dtype: bool


In [ ]:
df.loc[0, ["Text", "File", "Folder"]]

Text      Please print and put with other stuff you just...
File                                                 1554.0
Folder                                               mann-k
Name: 0, dtype: object

In [ ]:
df["Email Length"] = email_length
df["Digit Density"] = digit_density
df["Space Density"] = space_density
df["Number of Paragraphs"] = number_paragraphs
df["Average Sentences per Paragraph"] = average_sentences_paragraph
df["Farewell Words"] = farewell_words
df["Freq Punc"] = freq_punc
df["Last Punc"] = last_punc
df["Average Word Length"] = avg_len
df["Average Sentence Length"] = avg_sent_len
df["Short Word Ratio"] = short_word_ratio
df["Punc Frequency"] = punc_freq
df["Punc after Greeting"] = punc_greet
df["Number Words"] = number_words
df["Subjectivity"] = subjectivity
df["Polarity"] = polarity
df["Most Common POS"] = pos
df["Single Sentence"] = single_sentence
df["Greeting"] = greeting_words
df["Most Common Word"] = most_common_word
df["Freq Most Common Word"] = freq_most_common_word
df["Total Special Character Count"] = number_special_characters
df["Max Occurring Special Char"] = max_special_character
df["Count of Max Special Char"] = freq_max_special_character

In [ ]:
df

,Author,Folder,File,Text,Raw Text,Message ID,Email Length,Digit Density,Space Density,Number of Paragraphs,...,Subjectivity,Polarity,Most Common POS,Single Sentence,Greeting,Most Common Word,Freq Most Common Word,Total Special Character Count,Max Occurring Special Char,Count of Max Special Char
0,Kay,mann-k,1554.0,Please print and put with other stuff you just...,Message-ID: <11319742.1075845955870.JavaMail.e...,11319742.1075845955870.,51,0.000000,0.235294,1.0,...,0.287500,0.037500,NN,True,Please,Please,1,0,None,0
1,Kay,mann-k,2692.0,OPPS. I registered twice. I'm just taking up...,Message-ID: <6446758.1075845983982.JavaMail.ev...,6446758.1075845983982.,48,0.000000,0.312500,1.0,...,0.000000,0.000000,PRP,False,OPPS,I,2,0,None,0
2,Kay,mann-k,1904.0,"Hi Sheila,\n\nI'm not up on this part of the d...",Message-ID: <31988618.1075845964396.JavaMail.e...,31988618.1075845964396.,37,0.000000,0.378378,2.0,...,0.000000,0.000000,NN,True,Hi,Hi,1,0,None,0
3,Kay,mann-k,1993.0,Here's my stab at it:\n\n,Message-ID: <28515105.1075846076799.JavaMail.e...,28515105.1075846076799.,15,0.000000,0.400000,1.0,...,0.000000,0.000000,RB,True,Here,Here,1,0,None,0
4,Kay,mann-k,3494.0,"Hi Karen,\n\nI'm looking at a version which wa...",Message-ID: <29986588.1075846114096.JavaMail.e...,29986588.1075846114096.,110,0.018182,0.309091,2.0,...,1.000000,0.000000,NNP,False,Hi,I,2,0,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,Sara,shackleton-s,2369.0,Kevin: This is set to close tomorrow (so far!...,Message-ID: <13622728.1075844575814.JavaMail.e...,13622728.1075844575814.,95,0.000000,0.305263,1.0,...,0.511111,0.191667,NN,False,Kevin,to,2,2,(,1
19996,Sara,shackleton-s,1278.0,Jeff: I have not heard a word from credit. W...,Message-ID: <25204828.1075844548975.JavaMail.e...,25204828.1075844548975.,56,0.000000,0.339286,1.0,...,0.000000,0.000000,NN,True,Jeff,Jeff,1,0,None,0
19997,"Shackleton, Sara",shackleton-s,87.0,\nScott: this has not been reviewed for accu...,Message-ID: <4926010.1075858807724.JavaMail.ev...,4926010.1075858807724.,122,0.188525,0.237705,1.0,...,1.000000,-0.750000,NNP,True,None,Scott,1,2,(,1
19998,Sara,shackleton-s,2238.0,fyi\n,Message-ID: <16414307.1075844572728.JavaMail.e...,16414307.1075844572728.,3,0.000000,0.333333,1.0,...,0.000000,0.000000,NN,True,fyi,fyi,1,0,None,0


In [ ]:
df.to_csv("Enron_29_Features.csv")
